In [ ]:
#pip install requests

In [ ]:
#pip install beautifulsoup4

In [ ]:
import os

def save_html_to_file1(html_content, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(str(html_content))

def save_html_to_file(html_content, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(html_content.decode('utf-8'))

###### 1. Establish a connection to the webpage - "https://www.themoviedb.org/movie" - and provide the following details

In [ ]:
# a1. Import the requests library
import requests

# a2. Formulate a get request to download the contents of the webpage
needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

response = requests.get(("https://www.themoviedb.org/movie"),headers = needed_headers)

# b. Verify the status code of the request and confirm that the request was executed appropriately
if response.status_code == 200:
    print("Request executed successfully.")
    
    # c. Print the contents of the page obtained from the response and save it in a variable
    page_content = response.content
    print(page_content.decode('utf-8'))
    save_html_to_file(page_content, "Desktop/page_content.html")

    # d. Infer the type of the variable created in part 1c and display the first 200 characters of the content from the server’s response
    print(f"Type of page_content: {type(page_content)}")
    print(f"First 200 characters of the content:\n{page_content[:200].decode('utf-8')}")
else:
    print(f"Error: {response.status_code}")

###### 2. Parse the content of HTML response using the BeautifulSoup library and execute the tasks specified in the guidelines mentioned below

In [ ]:
# a1. From the BeautifulSoup library (bs4) import the BeautifulSoup class
from bs4 import BeautifulSoup

# a2. Pass the contents of the webpage obtained from step 1c as an argument to create an instance of the BeautifulSoup class
html_content = page_content.decode('utf-8')

soup = BeautifulSoup(html_content, 'html.parser')

# b. Extract the title of the parsed web page content using an appropriate method or attribute of the document object created in part 2a
title = soup.title.string
print(f"Title of the webpage: {title}")

# c. Write a user defined function to generalize the task presented in Q2a to any URL that retrieves the content of the webpage. Your function should take a URL string as an input and return a correctly formulated BeautifulSoup instance as the output. In your function definition, ensure that appropriate exceptions are raised to the user (through status codes) if they pass in malformed/incorrect URLs. Write two test cases for your function - one with a working URL and another with an URL that gets a 404 response.
def create_beautifulsoup_instance(url):
    try:
        # Make a request to the URL
        response = requests.get(url, headers = needed_headers)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup

    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error: {err}")
    except requests.exceptions.RequestException as err:
        print(f"Request Error: {err}")
    except Exception as err:
        print(f"An unexpected error occurred: {err}")

# Test case with a working URL
url_working = "https://www.themoviedb.org/movie"
soup_working = create_beautifulsoup_instance(url_working)
print(soup_working.title.string)

# Test case with a URL that gets a 404 response
url_not_found = "https://www.themoviedb.org/movies"
soup_not_found = create_beautifulsoup_instance(url_not_found)
# The function will print the error message for a 404 response


###### 3. Extract the content of the webpage - https://www.themoviedb.org/movie - that hosts a current dated listing of popular movies.

In [ ]:
# a.Write a function call to the user defined function created in 2c with the url https://www.themoviedb.org/movie as an input and store the response in a variable
url_popular_movies = "https://www.themoviedb.org/movie"

soup_popular_movies = create_beautifulsoup_instance(url_popular_movies)

# b.Print the HTML content associated with the first movie displayed on the web page using appropriate HTML tags to access this listing on the object created in part 3a 
first_movie_html = soup_popular_movies.find('div', class_='page_wrap movie_wrap')
print(first_movie_html)
filename = "Desktop/first_movie_content.html"
save_html_to_file1(first_movie_html, filename)

# c.Display the name of the first movie using appropriate HTML tags to access this listing on the object created in part 3a
movie_name_found = False
if first_movie_html:
    column_movies = first_movie_html.find('div', class_="white_column no_pad")
    if column_movies:
        movie_content = column_movies.find('div', class_= "content")
        if movie_content:
            h2_tag = movie_content.find("h2")
            if h2_tag:
                movie_name_found = True
                print(f"Name of the first movie: {h2_tag.a.text}")

if movie_name_found == False:
    print("Unable to find the name of the first movie.")

    
# d.Display the user rating of the first movie by using appropriate HTML tags to access this listing on the object created in part 3a
rating_found = False
if movie_content:
    consensus_tag = movie_content.find('div', class_="consensus tight")
    if consensus_tag:
        outer_ring_tag = consensus_tag.find('div', class_="outer_ring")
        if outer_ring_tag:
            user_score_tag = outer_ring_tag.find('div', class_="user_score_chart 613a9325e272600063455943")
            if user_score_tag:
                rating_found = True
                rating = user_score_tag["data-percent"]
                print(f"Rating of the first movie: {rating} %")

if rating_found == False:
    print("Unable to find the rating of the first movie.")  

# e.For the first movie, extract the part of the url following the string “https://www.themoviedb.org/” using the appropriate HTML tags to extract this portion on the object created in part 3a
part_of_url = False
if first_movie_html:
    column_movies = first_movie_html.find('div', class_="white_column no_pad")
    if column_movies:
        movie_content = column_movies.find('div', class_= "content")
        if movie_content:
            h2_tag = movie_content.find("h2")
            if h2_tag:
                part_of_url = True
                part = h2_tag.a['href']
                print(f"Part of the URL: {part}")

if part_of_url == False:
    print("Unable to find the part of the url for the first movie.")  

###### 4. Write user defined functions for each subsection below (i.e., Q4 a, Q4b, Q4c, Q4d, and Q4e) to return

In [ ]:
import re
# a.Titles of all the movies on the page as a Python list
def get_movie_titles(soup):
    movie_titles = []
    first_movie_html = soup.find('div', class_='page_wrap movie_wrap')

    if first_movie_html:
        column_movies = first_movie_html.find('div', class_="white_column no_pad")
        if column_movies:
            for movie_content in column_movies.find_all('div', class_= "content"):
                if movie_content:
                    h2_tag = movie_content.find("h2")
                    if h2_tag:
                        movie_titles.append(h2_tag.a.text)

    return movie_titles

# b. User ratings of all the movies on the page as a Python list
def get_user_ratings(soup):
    user_ratings = []
    first_movie_html = soup.find('div', class_='page_wrap movie_wrap')

    if first_movie_html:
        column_movies = first_movie_html.find('div', class_="white_column no_pad")
        if column_movies:
            for movie_content in column_movies.find_all('div', class_= "content"):
                if movie_content:
                    user_score_tag = movie_content.find('div', class_=re.compile(r'^user_score_chart \d+'))
                    if user_score_tag:
                        user_ratings.append(user_score_tag["data-percent"])
                    else:
                        user_ratings.append("not rated")
    return user_ratings

# c. HTML content of all the individual pages of movies collected into a Python list
def get_html_contents(soup):
    html_contents = []
    first_movie_html = soup.find('div', class_='page_wrap movie_wrap')

    if first_movie_html:
        column_movies = first_movie_html.find('div', class_="white_column no_pad")
        if column_movies:
            for movie_content in column_movies.find_all('div', class_= "content"):
                if movie_content:
                    h2_tag = movie_content.find("h2")
                    if h2_tag:
                        part = h2_tag.a['href']
                        html_contents.append(part)

    return html_contents

# d. Genres of all the movies on the page as a Python list
def get_movie_genres(movie_html_content_list):
    genres_list = []
    for html_content in movie_html_content_list:
        soup = create_beautifulsoup_instance("https://www.themoviedb.org" + html_content)
        genres = [genre.text.strip().replace('\xa0', '') for genre in soup.find_all('span', class_='genres')]
        genres_list.append(genres)
    return genres_list

# e. Cast of all the movies on the page as a Python list
def get_movie_cast(movie_html_content_list):
    cast_list = []
    for html_content in movie_html_content_list:
        soup = create_beautifulsoup_instance("https://www.themoviedb.org" + html_content)
        cast_scroller = soup.find('div', class_='scroller_wrap should_fade is_fading')
        if cast_scroller:
            ordered_list = soup.find('ol', class_='people scroller')
            list_items = ordered_list.find_all('li')
            curr_list = []
            for item in list_items:
                img_tag = item.find('img')
                if img_tag:
                    cast = img_tag['alt']
                    curr_list.append(cast)
            cast_list.append(curr_list)
    return cast_list

In [ ]:
titles = get_movie_titles(soup_popular_movies)
print(titles)

ratings = get_user_ratings(soup_popular_movies)
print(ratings)

contents = get_html_contents(soup_popular_movies)
print(contents)
genres = get_movie_genres(contents)
print(genres)
casts = get_movie_cast(contents)
print(casts)

###### 5. Write an user defined function that returns a pandas data frame with following data: 

In [ ]:
import pandas as pd
def create_movie_dataframe(soup, contents):
    movie_data = []

    # a. Titles of the movies listed on the page
    movie_titles = get_movie_titles(soup)
    movie_data.append(movie_titles)
    
    # b. User ratings of the movies listed on the page
    user_ratings = get_user_ratings(soup)
    movie_data.append(user_ratings)
    
    # c. Genres of the movies listed on the page
    movie_genres = get_movie_genres(contents)
    movie_data.append(movie_genres)
    
    # d. Cast of the movies listed on the page
    movie_cast = get_movie_cast(contents)
    movie_data.append(movie_cast)

    df = pd.DataFrame(movie_data)
    return df

In [ ]:
print(create_movie_dataframe(soup_popular_movies, contents))

###### 6. Scraping the data and combining the dataframes

In [ ]:
# a.Write a function that scrapes data (mentioned in Q5) from page number 1, 2, 3, 4 and 5 on the URL https://www.themoviedb.org/movie and returns 5 data frames which can be exported to csv file by calling the functions defined in Q3a, Q4c and Q5 
def scrape_movies_from_pages(pages):
    all_dataframes = []
    for page in pages:
        url = f"https://www.themoviedb.org/movie?page={page}"
        soup = create_beautifulsoup_instance(url)
        contents = get_html_contents(soup)
        
        df = create_movie_dataframe(soup, contents)
        df.to_csv(f'Desktop/page_{page}_data.csv', index=False)
        all_dataframes.append(df)
    return all_dataframes

# b. Combine the data obtained from dataframes in Q6(a)
def combine_dataframes(dataframes):
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

In [ ]:
pages = [1, 2, 3, 4, 5]
all_dfs = scrape_movies_from_pages(pages)
df = combine_dataframes(all_dfs)

In [ ]:
print(df)